## Interaktiv Phi 3 Mini 4K Instruktionschatbot med Whisper

### Introduktion:
Den interaktive Phi 3 Mini 4K Instruktionschatbot er et værktøj, der giver brugere mulighed for at interagere med Microsoft Phi 3 Mini 4K instruktionsdemoen ved hjælp af tekst- eller lydinput. Chatbotten kan bruges til en række opgaver, såsom oversættelse, vejropdateringer og generel informationsindsamling.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


Opret din Huggingface Access Token

Opret en ny token  
Angiv et nyt navn  
Vælg skrive-tilladelser  
Kopiér token og gem den et sikkert sted


Den følgende Python-kode udfører to hovedopgaver: import af `os`-modulet og opsætning af en miljøvariabel.

1. Import af `os`-modulet:
   - `os`-modulet i Python giver en måde at interagere med operativsystemet på. Det gør det muligt at udføre forskellige opgaver relateret til operativsystemet, såsom adgang til miljøvariabler, arbejde med filer og mapper osv.
   - I denne kode importeres `os`-modulet ved hjælp af `import`-sætningen. Denne sætning gør funktionaliteten i `os`-modulet tilgængelig til brug i det aktuelle Python-script.

2. Opsætning af en miljøvariabel:
   - En miljøvariabel er en værdi, som kan tilgås af programmer, der kører på operativsystemet. Det er en måde at gemme konfigurationsindstillinger eller anden information, som kan bruges af flere programmer.
   - I denne kode sættes en ny miljøvariabel ved hjælp af `os.environ`-ordbogen. Nøglen i ordbogen er `'HF_TOKEN'`, og værdien tildeles fra variablen `HUGGINGFACE_TOKEN`.
   - Variablen `HUGGINGFACE_TOKEN` er defineret lige over denne kode, og den tildeles en strengværdi `"hf_**************"` ved hjælp af syntaksen `#@param`. Denne syntaks bruges ofte i Jupyter-notebooks til at tillade brugerinput og parameterkonfiguration direkte i notebook-grænsefladen.
   - Ved at sætte miljøvariablen `'HF_TOKEN'` kan den tilgås af andre dele af programmet eller andre programmer, der kører på det samme operativsystem.

Samlet set importerer denne kode `os`-modulet og sætter en miljøvariabel med navnet `'HF_TOKEN'` med den værdi, der er angivet i variablen `HUGGINGFACE_TOKEN`.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

Denne kodebid viser en funktion kaldet clear_output, som bruges til at rydde outputtet fra den aktuelle celle i Jupyter Notebook eller IPython. Lad os gennemgå koden og forstå dens funktionalitet:

Funktionen clear_output tager én parameter kaldet wait, som er en boolsk værdi. Som standard er wait sat til False. Denne parameter afgør, om funktionen skal vente, indtil nyt output er tilgængeligt for at erstatte det eksisterende output, før det ryddes.

Selve funktionen bruges til at rydde outputtet fra den aktuelle celle. I Jupyter Notebook eller IPython, når en celle genererer output, såsom udskrevet tekst eller grafiske plots, vises dette output under cellen. Funktionen clear_output giver dig mulighed for at fjerne dette output.

Implementeringen af funktionen er ikke angivet i kodebidet, som det fremgår af ellipsen (...). Ellipsen repræsenterer en pladsholder for den faktiske kode, der udfører rydningen af outputtet. Implementeringen af funktionen kan involvere interaktion med Jupyter Notebook- eller IPython-API'en for at fjerne det eksisterende output fra cellen.

Samlet set giver denne funktion en praktisk måde at rydde outputtet fra den aktuelle celle i Jupyter Notebook eller IPython, hvilket gør det lettere at administrere og opdatere det viste output under interaktive kodningssessioner.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Udfør tekst-til-tale (TTS) ved hjælp af Edge TTS-tjenesten. Lad os gennemgå de relevante funktionsimplementeringer én ad gangen:

1. `calculate_rate_string(input_value)`: Denne funktion tager en inputværdi og beregner hastighedsstrengen for TTS-stemmen. Inputværdien repræsenterer den ønskede talehastighed, hvor en værdi på 1 repræsenterer normal hastighed. Funktionen beregner hastighedsstrengen ved at trække 1 fra inputværdien, multiplicere den med 100 og derefter bestemme tegnet baseret på, om inputværdien er større end eller lig med 1. Funktionen returnerer hastighedsstrengen i formatet "{sign}{rate}".

2. `make_chunks(input_text, language)`: Denne funktion tager en inputtekst og et sprog som parametre. Den opdeler inputteksten i stykker baseret på sprogspecifikke regler. I denne implementering, hvis sproget er "English", opdeler funktionen teksten ved hver punktum (".") og fjerner eventuelle indledende eller afsluttende mellemrum. Den tilføjer derefter et punktum til hvert stykke og returnerer den filtrerede liste af stykker.

3. `tts_file_name(text)`: Denne funktion genererer et filnavn til TTS-lydfilen baseret på inputteksten. Den udfører flere transformationer på teksten: fjerner et afsluttende punktum (hvis til stede), konverterer teksten til små bogstaver, fjerner indledende og afsluttende mellemrum og erstatter mellemrum med understregninger. Den afkorter derefter teksten til maksimalt 25 tegn (hvis længere) eller bruger den fulde tekst, hvis den er tom. Til sidst genererer den en tilfældig streng ved hjælp af [`uuid`]-modulet og kombinerer den med den afkortede tekst for at skabe filnavnet i formatet "/content/edge_tts_voice/{truncated_text}_{random_string}.mp3".

4. `merge_audio_files(audio_paths, output_path)`: Denne funktion samler flere lydfiler i én enkelt lydfil. Den tager en liste over lydfilstier og en outputsti som parametre. Funktionen initialiserer et tomt `AudioSegment`-objekt kaldet [`merged_audio`]. Den itererer derefter gennem hver lydfilsti, indlæser lydfilen ved hjælp af metoden `AudioSegment.from_file()` fra `pydub`-biblioteket og tilføjer den aktuelle lydfil til [`merged_audio`]-objektet. Til sidst eksporterer den den sammensatte lyd til den angivne outputsti i MP3-format.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`: Denne funktion udfører TTS-operationen ved hjælp af Edge TTS-tjenesten. Den tager en liste over tekststykker, talehastigheden, stemmens navn og gemme-stien som parametre. Hvis antallet af stykker er større end 1, opretter funktionen en mappe til opbevaring af de individuelle lydfiler. Den itererer derefter gennem hvert stykke, konstruerer en Edge TTS-kommando ved hjælp af funktionen `calculate_rate_string()`, stemmens navn og tekststykket og udfører kommandoen ved hjælp af funktionen `os.system()`. Hvis kommandoens udførelse lykkes, tilføjer den stien til den genererede lydfil til en liste. Efter behandling af alle stykker samler den de individuelle lydfiler ved hjælp af funktionen `merge_audio_files()` og gemmer den sammensatte lyd til den angivne gemme-sti. Hvis der kun er ét stykke, genererer den direkte Edge TTS-kommandoen og gemmer lyden til gemme-stien. Til sidst returnerer den gemme-stien til den genererede lydfil.

6. `random_audio_name_generate()`: Denne funktion genererer et tilfældigt lydfilnavn ved hjælp af [`uuid`]-modulet. Den genererer en tilfældig UUID, konverterer den til en streng, tager de første 8 tegn, tilføjer ".mp3"-udvidelsen og returnerer det tilfældige lydfilnavn.

7. `talk(input_text)`: Denne funktion er hovedindgangspunktet for at udføre TTS-operationen. Den tager en inputtekst som parameter. Den kontrollerer først længden af inputteksten for at afgøre, om det er en lang sætning (større end eller lig med 600 tegn). Baseret på længden og værdien af variablen `translate_text_flag` bestemmer den sproget og genererer listen over tekststykker ved hjælp af funktionen `make_chunks()`. Den genererer derefter en gemme-sti til lydfilen ved hjælp af funktionen `random_audio_name_generate()`. Til sidst kalder den funktionen `edge_free_tts()` for at udføre TTS-operationen og returnerer gemme-stien til den genererede lydfil.

Samlet set arbejder disse funktioner sammen for at opdele inputteksten i stykker, generere et filnavn til lydfilen, udføre TTS-operationen ved hjælp af Edge TTS-tjenesten og samle de individuelle lydfiler i én enkelt lydfil.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

Implementeringen af to funktioner: convert_to_text og run_text_prompt, samt deklarationen af to klasser: str og Audio.

Funktionen convert_to_text tager en audio_path som input og transskriberer lyden til tekst ved hjælp af en model kaldet whisper_model. Funktionen kontrollerer først, om gpu-flaget er sat til True. Hvis det er tilfældet, bruges whisper_model med visse parametre såsom word_timestamps=True, fp16=True, language='English' og task='translate'. Hvis gpu-flaget er False, bruges whisper_model med fp16=False. Den resulterende transskription gemmes derefter i en fil kaldet 'scan.txt' og returneres som tekst.

Funktionen run_text_prompt tager en besked og en chat_history som input. Den bruger funktionen phi_demo til at generere et svar fra en chatbot baseret på inputbeskeden. Det genererede svar sendes derefter til funktionen talk, som konverterer svaret til en lydfil og returnerer filstien. Klassen Audio bruges til at vise og afspille lydfilen. Lyden vises ved hjælp af display-funktionen fra modulet IPython.display, og Audio-objektet oprettes med parameteren autoplay=True, så lyden starter automatisk. Chat_history opdateres med inputbeskeden og det genererede svar, og en tom streng samt den opdaterede chat_history returneres.

Klassen str er en indbygget klasse i Python, der repræsenterer en sekvens af tegn. Den tilbyder forskellige metoder til at manipulere og arbejde med strenge, såsom capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill og flere. Disse metoder gør det muligt at udføre operationer som søgning, erstatning, formatering og manipulation af strenge.

Klassen Audio er en brugerdefineret klasse, der repræsenterer et lydobjekt. Den bruges til at oprette en lydafspiller i Jupyter Notebook-miljøet. Klassen accepterer forskellige parametre såsom data, filename, url, embed, rate, autoplay og normalize. Parameteren data kan være en numpy-array, en liste af samples, en streng, der repræsenterer en filnavn eller URL, eller rå PCM-data. Parameteren filename bruges til at angive en lokal fil, hvorfra lyddataene skal indlæses, og parameteren url bruges til at angive en URL, hvorfra lyddataene skal downloades. Parameteren embed bestemmer, om lyddataene skal indlejres ved hjælp af en data-URI eller refereres fra den originale kilde. Parameteren rate angiver samplingsraten for lyddataene. Parameteren autoplay bestemmer, om lyden skal starte automatisk. Parameteren normalize angiver, om lyddataene skal normaliseres (omskaleres) til det maksimale mulige område. Klassen Audio tilbyder også metoder som reload til at genindlæse lyddataene fra fil eller URL, og attributter som src_attr, autoplay_attr og element_id_attr til at hente de tilsvarende attributter for lydelementet i HTML.

Samlet set bruges disse funktioner og klasser til at transskribere lyd til tekst, generere lydsvar fra en chatbot og vise samt afspille lyd i Jupyter Notebook-miljøet.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi er ikke ansvarlige for eventuelle misforståelser eller fejltolkninger, der opstår som følge af brugen af denne oversættelse.
